## set DB

- Chroma DB 를 통해 VectorStore 를 구축한다.
- 벡터 저장소 생성 from_texts

In [2]:
import torch
from load_embedding import load_embedding
from pdf_preprocessing import LegalText
from langchain_core.documents import Document
from langchain_chroma import Chroma
from glob import glob
from pdf_preprocessing import LegalText

filenames = glob('/Volumes/MINDB/24년/SW아카데미/LangEyE/App/src/crawling/files/laws/*.pdf')

# print(filenames[0])
# LegalText(filenames[0]).documents

law_docs = []
for filename in filenames:
    law_docs.extend(LegalText(filename).documents)
print(len(law_docs))

501


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

DB_PATH = "./chroma_db"

db = Chroma.from_documents(
    documents=law_docs, embedding=load_embedding(device),
    persist_directory=DB_PATH,
    collection_name="my_db"
)

/Volumes/MINDB/24년/SW아카데미/LangEyE/App/models/RAG/load_embedding.py:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/Users/minsuson/.pyenv/versions/3.10.9/envs/service-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/minsuson/.pyenv/versions/3.10.9/envs/service-env/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformer

ValueError: Expected metadata value to be a str, int, float or bool, got None which is a NoneType in upsert.

Try filtering complex metadata from the document using langchain_community.vectorstores.utils.filter_complex_metadata.

## DB 로드하기

In [ ]:
DB_PATH = "./chroma_db"
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=load_embedding(device),
    collection_name="my_db"
)

/Volumes/MINDB/24년/SW아카데미/LangEyE/App/models/RAG/load_embedding.py:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/Users/minsuson/.pyenv/versions/3.10.9/envs/service-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/minsuson/.pyenv/versions/3.10.9/envs/service-env/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformer

KeyboardInterrupt: 

### 유사도 검색

`similarity_search` 메서드는 Chroma 데이터베이스에서 유사도 검색을 수행합니다. 이 메서드는 주어진 쿼리와 가장 유사한 문서들을 반환합니다.

**매개변수**

- `query` (str): 검색할 쿼리 텍스트
- `k` (int, 선택적): 반환할 결과의 수. 기본값은 4입니다.
- `filter` (Dict[str, str], 선택적): 메타데이터로 필터링. 기본값은 None입니다.

**참고**

- `k` 값을 조절하여 원하는 수의 결과를 얻을 수 있습니다.
- `filter` 매개변수를 사용하여 특정 메타데이터 조건에 맞는 문서만 검색할 수 있습니다.
- 이 메서드는 점수 정보 없이 문서만 반환합니다. 점수 정보도 필요한 경우 `similarity_search_with_score` 메서드를 직접 사용하세요.

**반환값**

- `List[Document]`: 쿼리 텍스트와 가장 유사한 문서들의 리스트

- query 에서 Pattern 을 찾아서 filter 를 적용한다.

In [ ]:
# filter 사용
db.similarity_search(
    "TF IDF 에 대하여 알려줘", filter={"source": "data/nlp-keywords.txt"}, k=2
)